# Sample Script for Querying the Census
* ACS 5 Year Variables: https://api.census.gov/data/2018/acs/acs5/variables.html

### Tips
* Different survey questions have different total population denomenators

In [2]:
import pandas as pd
import requests
import numpy as np

## Parameters

In [3]:
url = 'https://api.census.gov/data'

## select survey year
year = '2018'

## select survey
dataset = 'acs/acs5'

## select variable codes and the alias you want to assign
## alias prefix determines the denominator of percentages
var_dict = {'NAME':'Tract_Name'
           , 'TRACT':'ct2010'
            #, 'ZCTA':'zipcode'
           , 'B02001_001E': 'demo_total_pop' # verified that this is the same denom for B03
           , 'B02009_001E': 'demo_black'
           , 'B03002_012E': 'demo_hispanic'
           , 'B29001_001E': 'age_total_pop'
           , 'B29001_005E': 'age_65_Plus'
           , 'B29001_004E': 'age_45_64'
           , 'B29001_003E': 'age_30_44'
            ## education columns are mutually exclusive
           , 'B06009_001E': 'edu_total_pop'
           , 'B06009_002E': 'edu_less_than_hs'
           , 'B06009_003E': 'edu_hs_grad'
           , 'B06009_004E': 'edu_some_college_or_associates'
           , 'B06009_005E': 'edu_bachelors'
           , 'B06009_006E': 'edu_graduate_deg'
           }

## select state fips code
# use '*' for all

geo_level = 'tract'
code_for_geo_level = '*'
geo_restriction_level = 'state'
code_geo_restriction_level = '36'

## Query

In [81]:
### query the API
var = ','.join([i for i in var_dict])

geo = ('&for={}:{}&in={}:{}'
       .format(geo_level
               , code_for_geo_level
               , geo_restriction_level
               , code_geo_restriction_level))

query = url + '/' +  year + '/' + dataset + '?get=' +  var + geo

resp = (requests
        .get(query)
        .json()
       )

## turn into dataframe
census_output = (pd.DataFrame(resp[1:],columns = resp[0])
                .rename(columns = var_dict))


## custom columns
census_output['age_Under_30'] = (census_output['age_total_pop'].astype(int)
                                - census_output['age_65_Plus'].astype(int)
                                - census_output['age_45_64'].astype(int)
                                - census_output['age_30_44'].astype(int))

census_output['edu_No_College'] = (census_output['edu_less_than_hs'].astype(int)
                                    + census_output['edu_hs_grad'].astype(int))


## percentages: makes sure that the right denomenator is used
census_output['demo_total_pop'] = census_output['demo_total_pop'].astype(int)
census_output['age_total_pop'] = census_output['age_total_pop'].astype(int)
for col in var_dict:
    if var_dict[col] not in ['demo_total_pop', 'Tract_Name', 'ct2010']:
        pop_var_name = '{}_total_pop'.format(var_dict[col].split('_')[0])
        census_output[var_dict[col]] = census_output[var_dict[col]].astype(int)
        census_output['pct_' + var_dict[col]] = (census_output[var_dict[col]] 
                                                 / census_output[pop_var_name])
        
for col in ['age_Under_30', 'edu_No_College']:
    pop_var_name = '{}_total_pop'.format(col.split('_')[0])
    census_output[col] = census_output[col].astype(int)
    census_output['pct_' + col] = (census_output[col]
                                                 / census_output[pop_var_name])


## assign nyc's counties
census_output['COUNTY'] = census_output['county'].apply(lambda x:
                                                        'Queens County' if x == '081'
                                                        else ('Kings County' if x == '047'
                                                        else ('Richmond County' if x == '085'
                                                        else ('Bronx County' if x == '005'
                                                        else ('New York County' if x == '061'
                                                            else np.nan
                                                        ))))
                                                       )

## only keep if tract in nyc county
census_output = census_output.loc[pd.notna(census_output.COUNTY)].reset_index(drop = True)

In [82]:
census_output.columns

Index(['Tract_Name', 'ct2010', 'demo_total_pop', 'demo_black', 'demo_hispanic',
       'age_total_pop', 'age_65_Plus', 'age_45_64', 'age_30_44',
       'edu_total_pop', 'edu_less_than_hs', 'edu_hs_grad',
       'edu_some_college_or_associates', 'edu_bachelors', 'edu_graduate_deg',
       'state', 'county', 'tract', 'age_Under_30', 'edu_No_College',
       'pct_demo_black', 'pct_demo_hispanic', 'pct_age_total_pop',
       'pct_age_65_Plus', 'pct_age_45_64', 'pct_age_30_44',
       'pct_edu_total_pop', 'pct_edu_less_than_hs', 'pct_edu_hs_grad',
       'pct_edu_some_college_or_associates', 'pct_edu_bachelors',
       'pct_edu_graduate_deg', 'pct_age_Under_30', 'pct_edu_No_College',
       'COUNTY'],
      dtype='object')

## Output

In [61]:
census_output.loc[census_output.demo_total_pop > 0]

,Tract_Name,ct2010,demo_total_pop,demo_black,demo_hispanic,age_total_pop,age_65_Plus,age_45_64,age_30_44,edu_total_pop,...,pct_age_30_44,pct_edu_total_pop,pct_edu_less_than_hs,pct_edu_hs_grad,pct_edu_some_college_or_associates,pct_edu_bachelors,pct_edu_graduate_deg,pct_age_Under_30,pct_edu_No_College,COUNTY
0,"Census Tract 37, New York County, New York",003700,2579,94,224,1862,362,601,577,2143,...,0.309882,1.0,0.040131,0.047130,0.131125,0.403640,0.377975,0.172932,0.087261,New York County
1,"Census Tract 86.01, New York County, New York",008601,3022,72,131,2234,224,361,558,2290,...,0.249776,1.0,0.007424,0.069432,0.061572,0.486463,0.375109,0.488362,0.076856,New York County
2,"Census Tract 44, New York County, New York",004400,16684,1191,1315,13170,2563,3824,2783,12490,...,0.211314,1.0,0.022818,0.078062,0.113291,0.417774,0.368054,0.303721,0.100881,New York County
3,"Census Tract 86.03, New York County, New York",008603,4313,138,250,3479,1605,981,563,3691,...,0.161828,1.0,0.008941,0.041181,0.097805,0.386074,0.465998,0.094855,0.050122,New York County
5,"Census Tract 138, Queens County, New York",013800,3759,166,1152,2136,376,724,664,2643,...,0.310861,1.0,0.099886,0.232312,0.263715,0.262202,0.141884,0.174157,0.332198,Queens County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,"Census Tract 1502, Kings County, New York",150200,2811,100,339,1983,268,619,724,2008,...,0.365103,1.0,0.077689,0.121016,0.194721,0.361554,0.245020,0.187595,0.198705,Kings County
2161,"Census Tract 1522, Kings County, New York",152200,3737,768,576,2246,441,639,714,2529,...,0.317898,1.0,0.085409,0.135231,0.126137,0.337683,0.315540,0.201247,0.220641,Kings County
2163,"Census Tract 244.02, Richmond County, New York",024402,4654,33,191,3339,487,1471,674,3082,...,0.201857,1.0,0.072031,0.252433,0.201168,0.255678,0.218689,0.211740,0.324465,Richmond County
2164,"Census Tract 277.05, Richmond County, New York",027705,5843,211,440,4087,1054,1318,810,4240,...,0.198189,1.0,0.126415,0.353066,0.194340,0.166509,0.159670,0.221434,0.479481,Richmond County


## Export

In [83]:
#census_output.to_csv('census_by_tract.csv', index = False)

# Other

In [57]:
pd.read_stata('/Users/JoshCoven/Dropbox/Covid-19 Research Project/data/final/merged_nyc.dta').columns

Index(['zipcodetabulationarea', 'covered_share', 'private_share',
       'public_share', 'workhome_share', 'publictran_share',
       'commutetime_mean', 'tot_emp', 'cat_essential_prof_share',
       'cat_nonessential_prof_share', 'cat_science_share', 'cat_law_share',
       'cat_health_share', 'cat_health_other_share', 'cat_firefighting_share',
       'cat_law_enforce_share', 'cat_essential_service_share',
       'cat_nonessential_service_share', 'cat_industrial_share',
       'cat_essential_tech_share', 'cat_transportation_share', 'hisp_share',
       'asian_share', 'black_share', 'white_share', 'pop_density', 'hh_size',
       'male_share', 'share_20_40', 'share_40_60', 'share_60_plus'],
      dtype='object')

# County Level

In [66]:
var_dict = {'NAME':'County_Name'
           , 'B02001_001E': 'demo_total_pop' # verified that this is the same denom for B03
           , 'B02009_001E': 'demo_black'
           , 'B03002_012E': 'demo_hispanic'
           , 'B29001_001E': 'age_total_pop'
           , 'B29001_005E': 'age_65_Plus'
           , 'B29001_004E': 'age_45_64'
           , 'B29001_003E': 'age_30_44'
            ## education columns are mutually exclusive
           , 'B06009_001E': 'edu_total_pop'
           , 'B06009_002E': 'edu_less_than_hs'
           , 'B06009_003E': 'edu_hs_grad'
           , 'B06009_004E': 'edu_some_college_or_associates'
           , 'B06009_005E': 'edu_bachelors'
           , 'B06009_006E': 'edu_graduate_deg'
           }

## select state fips code
# use '*' for all

geo_level = 'county'
code_for_geo_level = '*'
geo_restriction_level = 'state'
code_geo_restriction_level = '*'


### query the API
var = ','.join([i for i in var_dict])

geo = ('&for={}:{}&in={}:{}'
       .format(geo_level
               , code_for_geo_level
               , geo_restriction_level
               , code_geo_restriction_level))



query = url + '/' +  year + '/' + dataset + '?get=' +  var + geo

resp = (requests
        .get(query)
        .json()
       )


## turn into dataframe
census_output = (pd.DataFrame(resp[1:],columns = resp[0])
                .rename(columns = var_dict))

## percentages: makes sure that the right denomenator is used
census_output['demo_total_pop'] = census_output['demo_total_pop'].astype(int)
census_output['age_total_pop'] = census_output['age_total_pop'].astype(int)
for col in var_dict:
    if var_dict[col] not in ['demo_total_pop', 'Tract_Name', 'County_Name', 'ct2010']:
        pop_var_name = '{}_total_pop'.format(var_dict[col].split('_')[0])
        census_output[var_dict[col]] = census_output[var_dict[col]].astype(float)
        census_output['pct_' + var_dict[col]] = (census_output[var_dict[col]] 
                                                 / census_output[pop_var_name])

## custom columns
census_output['pct_age_Under_30'] = (1 
                                - census_output['pct_age_65_Plus'] 
                                - census_output['pct_age_45_64'] 
                                - census_output['pct_age_30_44'])

census_output['pct_edu_No_College'] = (census_output['pct_edu_less_than_hs']
                                        + census_output['pct_edu_hs_grad'])


# Export

In [70]:
#census_output.to_csv('census_by_county.csv', index = False)

# ZIP level

In [ ]:
'https://api.census.gov/data/2017/acs/acs5?get=NAME,group(B19013)&for=zip%20code%20tabulation%20area:*'
'https://api.census.gov/data/2017/acs/acs5?get=NAME,B02001_001E&for=zip%20code%20tabulation%20area:*'
'https://api.census.gov/data/2017/acs/acs5?get=NAME,B02001_001E&for=zip%20code%20%tabulation%20area:*'

In [43]:
query

'https://api.census.gov/data/2017/acs/acs5?get=NAME,B02001_001E,B29001_001E&for=zip%20code%20tabulation%20area:*'

In [84]:
var_dict = {'NAME':'ZIP'
           , 'B02001_001E': 'demo_total_pop' # verified that this is the same denom for B03
           , 'B02009_001E': 'demo_black'
           , 'B03002_012E': 'demo_hispanic'
           , 'B29001_001E': 'age_total_pop'
            
            
            

            #, 'B01001_027E': 'age_f_Under_5'
            #, 'B01001_028E' : 'age_f_5_9'           
            #, 'B01001_029E': 'age_f_5_9'           
            #, 'B01001_030E': 'age_f_5_9' 
            #, 'B01001_031E': 'age_f_5_9' 
            #, 'B01001_032E': 'age_f_5_9' 
            #, 'B01001_033E': 'age_f_5_9' 
            #, 'B01001_034E': 'age_f_5_9' 
            #, 'B01001_035E': 'age_f_5_9' 
            
           #, 'B29001_005E': 'age_65_Plus'
           #, 'B29001_004E': 'age_45_64'
           #, 'B29001_003E': 'age_30_44'
            ## education columns are mutually exclusive
           , 'B06009_001E': 'edu_total_pop'
           , 'B06009_002E': 'edu_less_than_hs'
           , 'B06009_003E': 'edu_hs_grad'
           , 'B06009_004E': 'edu_some_college_or_associates'
           , 'B06009_005E': 'edu_bachelors'
           , 'B06009_006E': 'edu_graduate_deg'
            , 'group(B01001)':'test'
            }

## select state fips code
# use '*' for all

geo_level = 'zip%20code%20tabulation%20area'
code_for_geo_level = '*'
geo_restriction_level = 'state'
code_geo_restriction_level = '36'
year = '2018'

### query the API
var = ','.join([i for i in var_dict])

geo = ('&for={}:{}'
       .format(geo_level
               , code_for_geo_level
               , geo_restriction_level
               , code_geo_restriction_level))



query = url + '/' +  year + '/' + dataset + '?get=' +  var + geo

resp = (requests
        .get(query)
        .json()
       )


## turn into dataframe
census_output = (pd.DataFrame(resp[1:],columns = resp[0])
                .rename(columns = var_dict))

## percentages: makes sure that the right denomenator is used
census_output['demo_total_pop'] = pd.to_numeric(census_output['demo_total_pop']
                                               , errors = 'coerce')
#census_output['age_total_pop'] = census_output['age_total_pop'].astype(int)
census_output['edu_total_pop'] = pd.to_numeric(census_output['edu_total_pop']
                                               , errors = 'coerce')
for col in var_dict:
    if var_dict[col] not in ['ZIP','demo_total_pop', 'Tract_Name', 'County_Name', 'ct2010']:
        pop_var_name = '{}_total_pop'.format(var_dict[col].split('_')[0])
        census_output[var_dict[col]] = census_output[var_dict[col]].astype(float)
        census_output[var_dict[col]] = pd.to_numeric(census_output[var_dict[col]]
                                               , errors = 'coerce')
        census_output['pct_' + var_dict[col]] = (census_output[var_dict[col]] 
                                                 / census_output[pop_var_name])

## custom columns
#census_output['pct_age_Under_30'] = (1 
#                                - census_output['pct_age_65_Plus'] 
 #                               - census_output['pct_age_45_64'] 
 #                               - census_output['pct_age_30_44'])

census_output['pct_edu_No_College'] = (census_output['pct_edu_less_than_hs']
                                        + census_output['pct_edu_hs_grad'])

census_output['ZIP'] = census_output['ZIP'].apply(lambda x: x[-5:])

KeyError: 'test'

In [88]:
census_output#.loc[census_output.ZIP == '11366']
census_output.columns.tolist()

['ZIP',
 'demo_total_pop',
 'demo_black',
 'demo_hispanic',
 'age_total_pop',
 'age_65_Plus',
 'edu_total_pop',
 'edu_less_than_hs',
 'edu_hs_grad',
 'edu_some_college_or_associates',
 'edu_bachelors',
 'edu_graduate_deg',
 'GEO_ID',
 'B01001_001E',
 'B01001_001M',
 'B01001_002E',
 'B01001_002M',
 'B01001_003E',
 'B01001_003M',
 'B01001_004E',
 'B01001_004M',
 'B01001_005E',
 'B01001_005M',
 'B01001_006E',
 'B01001_006M',
 'B01001_007E',
 'B01001_007M',
 'B01001_008E',
 'B01001_008M',
 'B01001_009E',
 'B01001_009M',
 'B01001_010E',
 'B01001_010M',
 'B01001_011E',
 'B01001_011M',
 'B01001_012E',
 'B01001_012M',
 'B01001_013E',
 'B01001_013M',
 'B01001_014E',
 'B01001_014M',
 'B01001_015E',
 'B01001_015M',
 'B01001_016E',
 'B01001_016M',
 'B01001_017E',
 'B01001_017M',
 'B01001_018E',
 'B01001_018M',
 'B01001_019E',
 'B01001_019M',
 'B01001_020E',
 'B01001_020M',
 'B01001_021E',
 'B01001_021M',
 'B01001_022E',
 'B01001_022M',
 'B01001_023E',
 'B01001_023M',
 'B01001_024E',
 'B01001_024M'

# Export

In [71]:
#census_output.to_csv('census_zip_level.csv', index = False)

In [98]:
census_output[['ZIP', 'demo_total_pop',

 'B01001_002E',

 
 'B01001_026E',
]]



,ZIP,ZIP,demo_total_pop,B01001_002E,B01001_026E
0,ZCTA5 00601,ZCTA5 00601,17242,8426,8816
1,ZCTA5 00602,ZCTA5 00602,38442,18842,19600
2,ZCTA5 00603,ZCTA5 00603,48814,23939,24875
3,ZCTA5 00606,ZCTA5 00606,6437,3212,3225
4,ZCTA5 00610,ZCTA5 00610,27073,13112,13961
...,...,...,...,...,...
33115,ZCTA5 99923,ZCTA5 99923,15,0,15
33116,ZCTA5 99925,ZCTA5 99925,927,526,401
33117,ZCTA5 99926,ZCTA5 99926,1635,882,753
33118,ZCTA5 99927,ZCTA5 99927,38,20,18


In [96]:
census_output[variables]

,ZIP,ZIP,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,...,B01001_040E,B01001_041E,B01001_042E,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E
0,ZCTA5 00601,ZCTA5 00601,8426,428,479,659,342,183,92,78,...,637,516,309,393,188,318,389,263,230,203
1,ZCTA5 00602,ZCTA5 00602,18842,853,1249,872,767,533,255,405,...,1532,1446,560,977,473,547,1066,644,431,350
2,ZCTA5 00603,ZCTA5 00603,23939,1149,1363,1687,1103,759,265,330,...,1708,1506,638,1114,597,1134,1346,895,731,551
3,ZCTA5 00606,ZCTA5 00606,3212,146,143,203,140,86,69,72,...,265,184,152,153,115,131,113,70,46,132
4,ZCTA5 00610,ZCTA5 00610,13112,565,557,1049,566,374,126,209,...,1039,987,252,616,393,556,702,663,290,287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,ZCTA5 99923,ZCTA5 99923,0,0,0,0,0,0,0,0,...,0,0,0,0,0,15,0,0,0,0
33116,ZCTA5 99925,ZCTA5 99925,526,42,25,51,17,12,5,9,...,33,50,13,8,7,11,30,8,1,5
33117,ZCTA5 99926,ZCTA5 99926,882,61,87,74,43,5,5,26,...,52,71,25,13,19,5,50,13,6,5
33118,ZCTA5 99927,ZCTA5 99927,20,0,0,0,0,0,0,0,...,9,9,0,0,0,0,0,0,0,0


In [97]:
variables

['ZIP',
 'B01001_002E',
 'B01001_003E',
 'B01001_004E',
 'B01001_005E',
 'B01001_006E',
 'B01001_007E',
 'B01001_008E',
 'B01001_009E',
 'B01001_010E',
 'B01001_011E',
 'B01001_012E',
 'B01001_013E',
 'B01001_014E',
 'B01001_015E',
 'B01001_016E',
 'B01001_017E',
 'B01001_018E',
 'B01001_019E',
 'B01001_020E',
 'B01001_021E',
 'B01001_022E',
 'B01001_023E',
 'B01001_024E',
 'B01001_025E',
 'B01001_026E',
 'B01001_027E',
 'B01001_028E',
 'B01001_029E',
 'B01001_030E',
 'B01001_031E',
 'B01001_032E',
 'B01001_033E',
 'B01001_034E',
 'B01001_035E',
 'B01001_036E',
 'B01001_037E',
 'B01001_038E',
 'B01001_039E',
 'B01001_040E',
 'B01001_041E',
 'B01001_042E',
 'B01001_043E',
 'B01001_044E',
 'B01001_045E',
 'B01001_046E',
 'B01001_047E',
 'B01001_048E',
 'B01001_049E']